# Risk Model Score

This notebook reads and combines EAGLE-I outage data from 2022 and 2023.

In [1]:
import pandas as pd

In [2]:
# Read the CSV files
outage_2022 = pd.read_csv('eaglei_outage/eaglei_outages_with_events_2022.csv')
outage_2023 = pd.read_csv('eaglei_outage/eaglei_outages_with_events_2023.csv')

In [ ]:
# Union (concatenate) the two dataframes
outage = pd.concat([outage_2022, outage_2023], ignore_index=True)

print(f"Combined data shape: {outage.shape}")
outage.head()

Combined data shape: (94853, 14)


,event_id,state_event,Datetime Event Began,Datetime Restoration,Event Type,fips,state,county,start_time,duration,end_time,min_customers,max_customers,mean_customers
0,Alabama-0,Alabama,2022-01-03 01:00:00,2022-01-03 14:00:00,Severe Weather,1001,Alabama,Autauga,2022-01-03 04:00:00,5.75,2022-01-03 09:45:00,452,723,583.478261
1,Alabama-0,Alabama,2022-01-03 01:00:00,2022-01-03 14:00:00,Severe Weather,1003,Alabama,Baldwin,2022-01-03 13:45:00,3.00,2022-01-03 16:45:00,738,1404,1096.666667
2,Alabama-0,Alabama,2022-01-03 01:00:00,2022-01-03 14:00:00,Severe Weather,1005,Alabama,Barbour,2022-01-03 03:45:00,11.50,2022-01-03 15:15:00,208,979,570.826087
3,Alabama-0,Alabama,2022-01-03 01:00:00,2022-01-03 14:00:00,Severe Weather,1007,Alabama,Bibb,2022-01-03 03:00:00,2.50,2022-01-03 05:30:00,252,254,252.400000
4,Alabama-0,Alabama,2022-01-03 01:00:00,2022-01-03 14:00:00,Severe Weather,1009,Alabama,Blount,2022-01-03 03:45:00,2.00,2022-01-03 05:45:00,295,299,296.875000


In [ ]:
# Convert datetime columns to proper datetime type
outage['start_time'] = pd.to_datetime(outage['start_time'])
# outage['end_time'] = pd.to_datetime(outage['end_time'])

In [ ]:
outage_filtered = outage[(outage['state_event'] == 'California') & 
                         (outage['Event Type'] == 'Severe Weather') & 
                         (outage['start_time'] >= '2022-12-01') & 
                         (outage['start_time'] <= '2023-03-31')]

In [ ]:
outage_agg = outage_filtered.groupby([outage_filtered['start_time'].dt.date, 'county']).agg({
     'duration': 'sum',
     'max_customers': 'max'
     }).reset_index()

In [ ]:
outage_agg['risk score'] = 1

In [ ]:
outage_agg.to_csv('risk_model_y.csv', index=False)
outage_agg